In [13]:

import requests
from PIL import Image
from io import BytesIO
import boto3
import os, dotenv

dotenv.load_dotenv()

True

In [ ]:
# Initialize Connection Stringss
aws_access_key_id = os.getenv('AWS_SECRET_PUBLIC_KEY')
aws_secret_access_key = os.getenv('AWS_SECRET_PRIVATE_KEY')
aws_bucket_name = os.getenv('AWS_BUCKET_NAME')
connection_string = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_IP')}/{os.getenv('POSTGRES_DB')}"
print(aws_access_key_id, aws_secret_access_key, aws_bucket_name)
print(connection_string)

## Initiate AWS-S3 Session

In [34]:
# Initiate AWS & S3 Session
aws_session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_SECRET_PUBLIC_KEY'),
    aws_secret_access_key=os.getenv('AWS_SECRET_PRIVATE_KEY')
)
aws_s3_session = aws_session.resource('s3')
aws_session_bucket = aws_s3_session.Bucket(aws_bucket_name)

In [38]:
aws_s3_session.Bucket(aws_bucket_name).put_object(Key='images/test.txt', Body=b'Hello World!')

s3.Object(bucket_name='whereisthegooglecar-cdn', key='images/test.txt')

## Initialize Database

In [40]:
import asyncpg
database = await asyncpg.create_pool(dsn=connection_string)

In [43]:
result = await database.fetch('SELECT * FROM spottings')
image_urls = [row['imageUrl'] for row in result]
image_urls

['images/1213505199222685766.webp',
 'images/1212960828903653386.webp',
 'images/1212923305120763904.webp',
 'images/1212855100352241674.webp',
 'images/1212747471021670430.webp',
 'images/1212740222765502544.webp',
 'images/1212386119489028187.webp',
 'images/1212299947404300288.webp',
 'images/1211951648071618590.webp',
 'images/1211592468353781800.webp',
 'images/1211591115573170207.webp',
 'images/1211586007481323581.webp',
 'images/1211113305423216720.webp',
 'images/1211110435521306624.webp',
 'images/1210787810668318720.webp',
 'images/1210787198396399748.webp',
 'images/1210786673026273301.webp',
 'images/1210786495904878612.webp',
 'images/1210784959132991488.webp',
 'images/1210768725259591710.webp',
 'images/1210715579342655518.webp',
 'images/1210357099507486730.webp',
 'images/1210356107122577461.webp',
 'images/1210053761066868868.webp',
 'images/1210052096196481044.webp',
 'images/1210045392381808641.webp',
 'images/1209483899172818984.webp',
 'images/1208514017094541323

## Download & Optimize Images

In [66]:
import os, tempfile
# os.mkdir('images')

In [69]:
print(len(image_urls))
for image in image_urls:
    image_loc = image
    print(image_urls.index(image) + 1, image)

    image_data = aws_s3_session.Object(aws_bucket_name, image).get()['Body'].read()
    image_io = BytesIO(image_data)
    image = Image.open(image_io)

    # Optimize Image
    # create temp file to append to IO
    with tempfile.NamedTemporaryFile(delete=False) as temp:
        image.save(temp.name, format="WebP", quality=10, optimize=True)        
        temp.close()

        with open(temp.name, 'rb') as optimized:
            optimized_image_io = BytesIO(optimized.read())
    os.remove(temp.name)
    
    # Grab Data from IO
    data = optimized_image_io.getvalue()
    
    # Save to Storage
    # with open(image_loc, 'wb') as f:
    #     f.write(data)
        
    # Uploads to S3
    aws_s3_session.Bucket(aws_bucket_name).put_object(Key=f'{image}', Body=data)
    # print(f"Optimized {image} to S3 on {aws_bucket_name}")
    
    # break

5986
1 images/1213505199222685766.webp
2 images/1212960828903653386.webp
3 images/1212923305120763904.webp
4 images/1212855100352241674.webp
5 images/1212747471021670430.webp
6 images/1212740222765502544.webp
7 images/1212386119489028187.webp
8 images/1212299947404300288.webp
9 images/1211951648071618590.webp
10 images/1211592468353781800.webp
11 images/1211591115573170207.webp
12 images/1211586007481323581.webp
13 images/1211113305423216720.webp
14 images/1211110435521306624.webp
15 images/1210787810668318720.webp
16 images/1210787198396399748.webp
17 images/1210786673026273301.webp
18 images/1210786495904878612.webp
19 images/1210784959132991488.webp
20 images/1210768725259591710.webp
21 images/1210715579342655518.webp
22 images/1210357099507486730.webp
23 images/1210356107122577461.webp
24 images/1210053761066868868.webp
25 images/1210052096196481044.webp
26 images/1210045392381808641.webp
27 images/1209483899172818984.webp
28 images/1208514017094541323.webp
29 images/12079991289914